In [5]:
import torch

W = 1024
W_PRIME  = 128

D = 49


original_tensor = torch.randn(1, W, D)
# TODO pad oroiignal tensor to be a multiple of W_PRIME
padding = W_PRIME - (W % W_PRIME) if W % W_PRIME != 0 else 0
padded_original_tensor = torch.nn.functional.pad(original_tensor, (0, 0, 0, padding))
masking_top = torch.full((original_tensor.shape[1],), False, dtype=torch.bool)
masking_bottom = torch.full((padding,), True, dtype=torch.bool)
masking = torch.cat((masking_top, masking_bottom))
print(padded_original_tensor.shape)
print(masking.shape)

torch.Size([1, 1024, 49])
torch.Size([1024])


In [6]:
noise = torch.rand(padded_original_tensor.shape[1])  # noise in [0, 1]

# sort noise for each sample
ids_shuffle = torch.argsort(noise)
ids_shuffle = torch.arange(W)
ids_restore = torch.argsort(ids_shuffle)
print(ids_shuffle.shape, ids_shuffle)
print(ids_restore.shape, ids_restore)

torch.Size([1024]) tensor([804, 689, 865,  ..., 676,  27, 523])
torch.Size([1024]) tensor([952, 797, 318,  ..., 199, 608, 975])


In [7]:
original_tensor_picked = original_tensor[:, ids_shuffle, :]
mask_picked = masking[ids_shuffle]
print(original_tensor_picked.shape)
print(ids_shuffle.shape)
print(mask_picked.shape)

torch.Size([1, 1024, 49])
torch.Size([1024])
torch.Size([1024])


In [8]:
batched_tensor = original_tensor_picked.reshape(-1, W_PRIME, D)
batched_masking = mask_picked.reshape(-1, W_PRIME)
print(batched_tensor.shape)
print(batched_masking.shape)

torch.Size([8, 128, 49])
torch.Size([8, 128])


In [10]:
unbatched_tensor = batched_tensor.reshape(1, -1, batched_tensor.shape[-1])

padded_ubatched_tensor = unbatched_tensor[:, ids_restore, :]
backtransformed_original_tensor = padded_ubatched_tensor[:, ~masking, :]
print(backtransformed_original_tensor.shape)
print(torch.all(backtransformed_original_tensor == original_tensor))

torch.Size([1, 1024, 49])
tensor(True)


In [49]:
1025 % W_PRIME

1

In [6]:
import torch
import math

# Generate a random angle around the Z axis
theta = torch.tensor([math.pi/2])

# Construct the rotation matrix
cos_theta = torch.cos(theta)
sin_theta = torch.sin(theta)
rot_matrix = torch.tensor([[cos_theta, -sin_theta, 0],
                           [sin_theta, cos_theta, 0],
                           [0, 0, 1]])

# Load your points matrix into a PyTorch tensor
points = torch.tensor([[1.0, 1.0, 1.0], [4, 5, 6], [7, 8, 9]])

# Apply the rotation to all points
rotated_points = torch.matmul(points, rot_matrix)

# Print the results
print("Original points:\n", points)
print("Rotation angle (in radians):", theta.item())
print("Rotation matrix:\n", rot_matrix)
print("Rotated points:\n", rotated_points)

Original points:
 tensor([[1., 1., 1.],
        [4., 5., 6.],
        [7., 8., 9.]])
Rotation angle (in radians): 1.5707963705062866
Rotation matrix:
 tensor([[-4.3711e-08, -1.0000e+00,  0.0000e+00],
        [ 1.0000e+00, -4.3711e-08,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  1.0000e+00]])
Rotated points:
 tensor([[ 1.0000, -1.0000,  1.0000],
        [ 5.0000, -4.0000,  6.0000],
        [ 8.0000, -7.0000,  9.0000]])
